In [1]:
pip install datasets


In [11]:
import pandas as pd
from transformers import TFAutoModel , AutoTokenizer
from datasets import load_dataset
from transformers import DistilBertTokenizer

In [3]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
input = tokenizer(['HR Dashboard'], padding = True, truncation = True,
                  return_tensors ='tf')
input

{'input_ids': <tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[  101, 17850, 24923,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[1, 1, 1, 1]], dtype=int32)>}

In [6]:
output = model(input)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 4, 768), dtype=float32, numpy=
array([[[-0.3039519 , -0.0305858 ,  0.00400882, ..., -0.4847506 ,
          0.34516576,  0.5188402 ],
        [ 0.35778362, -0.38408604,  0.37003654, ..., -0.61620265,
          0.59948844, -0.5405039 ],
        [-0.3738274 , -0.60607785, -0.2814977 , ...,  0.23797421,
          0.1242699 , -0.47484154],
        [ 0.9056071 , -0.07485908, -0.24208027, ...,  0.15136041,
         -0.7880484 , -0.16790864]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-0.92287546, -0.35723177, -0.3496863 ,  0.8278717 ,  0.16023648,
        -0.27237093,  0.95286185,  0.23496045, -0.16329418, -0.99997896,
        -0.42060584,  0.7907553 ,  0.9824214 ,  0.12147571,  0.9344821 ,
        -0.7458304 , -0.42436796, -0.6028366 ,  0.40586725, -0.8018198 ,
         0.7044959 ,  0.9990763 ,  0.33071578,  0.42010006,  0.43133718,
         0.89415425, -0.76

In [17]:
emotions = pd.read_csv('/content/layoffs.csv')

In [18]:
emotions

,location,industry,total_laid_off,percentage_laid_off,date,country
0,Hamburg,Consumer,400.0,NaN,1/11/2024,Germany
1,Tel Aviv,Consumer,300.0,0.10,1/11/2024,Israel
2,SF Bay Area,Consumer,170.0,0.17,1/11/2024,United States
3,Bengaluru,Marketing,125.0,0.05,1/11/2024,India
4,New York City,Media,100.0,0.05,1/11/2024,United States
...,...,...,...,...,...,...
3308,Los Angeles,Travel,NaN,1.00,3/16/2020,United States
3309,Los Angeles,Transportation,8.0,0.10,3/13/2020,United States
3310,SF Bay Area,Consumer,6.0,0.75,3/13/2020,United States
3311,Los Angeles,Retail,20.0,0.40,3/12/2020,United States


In [22]:
def tokenize(batch) :
    return tokenizer(batch["text"], padding=True, truncation=True)


In [36]:
# Load DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define a function to tokenize a single text
def tokenize_text(text):
    return tokenizer.encode_plus(text,
                                  add_special_tokens=True,
                                  max_length=512,
                                  padding='max_length',
                                  truncation=True,
                                  return_attention_mask=True,
                                  return_tensors='pt')

# Apply the function to each row of your DataFrame
emotions_encoded = emotions['country'].apply(tokenize_text)

# Now you have a Series of tokenized representations


In [33]:
emotions_encoded

0       [input_ids, attention_mask]
1       [input_ids, attention_mask]
2       [input_ids, attention_mask]
3       [input_ids, attention_mask]
4       [input_ids, attention_mask]
                   ...             
3308    [input_ids, attention_mask]
3309    [input_ids, attention_mask]
3310    [input_ids, attention_mask]
3311    [input_ids, attention_mask]
3312    [input_ids, attention_mask]
Name: country, Length: 3313, dtype: object

In [41]:
class BERTForClassification(layoffs.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)


AttributeError: 'DataFrame' object has no attribute 'keras'